### Import libraries for Audio Processing

In [1]:
#Python Libararies
import pyaudio
import os
import struct
import numpy as np
import time
from IPython.display import clear_output
import threading
import wave
import librosa
import io
import soundfile as sfts
from datetime import date
import datetime
from numpy import loadtxt
from keras.models import load_model

Using TensorFlow backend.


#### Set global variables for synchronization and looping

In [2]:
FLAG = True #Flag is set to use reading facility under particular lines of code. It will be used for synchnization.
RECORD_SECONDS = 2 #record data for these much seconds
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second
fileno = 1

#### Function for recording

In [3]:
#Recoding Function
tgt= []
def record30():
    FLAG = False
    print(FLAG)
    RECORDFRAMES = []
    ts ="test.wav"
    Wave_write = wave.open(ts,"wb")
    Wave_write.setframerate(44100)
    Wave_write.setnchannels(1)
    Wave_write.setsampwidth(2)
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        RECORDEDDATA = stream.read(CHUNK)
        Wave_write.writeframesraw(RECORDEDDATA)
        #RECORDFRAMES.append(RECORDEDDATA)
    #Wave_write.writeframesraw(bytes(RECORDFRAMES))
    y, sr = librosa.load(ts, sr = 44100)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
    to_append = f'{ts} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    #to_append += f' {tgt[1]}'
    print(len(to_append.split()[1:]))
    # load model
    model = load_model('model.h5')
    FLAG = True

#### Creating Instance of PYAUDIO for recording and processing

In [4]:
p = pyaudio.PyAudio()

# Generate Stream for Reading Audio
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

In [5]:
print("test")
window = np.blackman(CHUNK)
while True:
    try:
        if FLAG==True:
            data = stream.read(CHUNK) #Read Binary
        
        # unpack the data and times by the hamming window
        indata = np.array(wave.struct.unpack("%dh"%(len(data)/2),\
                                             data))*window
        # Take the fft and square each value
        fftData=abs(np.fft.rfft(indata))**2
        # find the maximum
        which = fftData[1:].argmax() + 1
        logsn = 10*np.log10(which)
        print(logsn)
        if logsn > 10 and FLAG==True:
            print("DB Value -- ", logsn, "FALG is:---", FLAG)
            record30()
            break
            clear_output(wait=True)
    except KeyboardInterrupt:
        print("press control-c again to quit")

test
0.0
0.0
4.771212547196624
3.010299956639812
4.771212547196624
4.771212547196624
3.010299956639812
4.771212547196624
3.010299956639812
0.0
4.771212547196624
4.771212547196624
3.010299956639812
4.771212547196624
0.0
0.0
0.0
0.0
3.010299956639812
0.0
3.010299956639812
0.0
0.0
3.010299956639812
0.0
0.0
0.0
3.010299956639812
0.0
4.771212547196624
0.0
0.0
6.020599913279624
3.010299956639812
0.0
0.0
3.010299956639812
0.0
0.0
3.010299956639812
4.771212547196624
3.010299956639812
0.0
3.010299956639812
0.0
3.010299956639812
0.0
0.0
0.0
0.0
9.030899869919436
8.450980400142567
4.771212547196624
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.010299956639812
0.0
0.0
0.0
0.0
0.0
3.010299956639812
0.0
3.010299956639812
12.30448921378274
DB Value --  12.30448921378274 FALG is:--- True
False
26
